In [3]:
import numpy as np
import time
import pandas as pd
from os import listdir
from os.path import isfile, isdir, join
from PIL import Image
import cv2
import math
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 50
mypath = "./Training"
size = 128

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) #確認有抓到GPU

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13627027981035449527
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1401988300
locality {
  bus_id: 1
  links {
  }
}
incarnation: 272300220540969152
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [5]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

tf.__version__ is 2.1.0
tf.keras.__version__ is: 2.2.4-tf


### 訓練集 img&label

#### 增加訓練集

In [7]:
image_gen_train = ImageDataGenerator(
    rescale=1./255,               # 從0~255整數，壓縮為0~1浮點數
    rotation_range=35,            # 隨機旋轉 ±45°
    width_shift_range=.15,        # 隨機水平移動 ±15%
    height_shift_range=.15,       # 隨機垂直移動 ±15%
    horizontal_flip=True,         # 隨機水平翻轉
    zoom_range=0.3                # 隨機縮放 50%
)
#image_gen_train.flow(img_tra,y_t,batch_size=64)

from PIL import Image
import keras
df_t = pd.read_excel("./Training-label.xlsx",header=None)  
k=180
for j in range(180):
    y_origin = df_t.loc[j,0]
    if j < 9:
         file_name = './Training/00'+str(j+1)+'.jpg'
    elif j < 99:
        file_name = './Training/0'+str(j+1)+'.jpg'
    else :
        file_name = './Training/'+str(j+1)+'.jpg'
    #print(file_name)
    img = cv2.imread(file_name)  
    img = cv2.resize(img, (size,size))##改變圖片尺寸
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Cv2讀進來是BGR，轉成RGB
    img_origin=img.copy()
    img= np.array(img, dtype=np.float32)
    img_combine=np.array([img,img,img,img],dtype=np.float32) ##輸入generator要是四維
    batch_gen = image_gen_train.flow(img_combine, batch_size=4)
    assert next(batch_gen).shape==(4, size, size, 3)
    
    for i,batch in enumerate(batch_gen):
        k=k+1
        if i>4 : break
        im_new = batch[0]
        #im_new  = np.array(im_new)
        #print(im_new.shape)
        im_new = keras.preprocessing.image.array_to_img(im_new)
        #im_new=Image.fromarray(im_new) # numpy轉image类
        im_new.save('./Training/'+str(k)+'.jpg') #顯示圖片
        #cv2.imwrite('./Training/'+str(200+i)+'.jpg',im_new)
        df_t = df_t.append([y_origin],ignore_index=True)

#### 訓練圖片組合成矩陣

In [8]:
##將cvs裡面答案轉為陣列
#df_t = df_t.T
df_temp = pd.get_dummies(df_t[0]) #One-hot編碼
y_t = np.array(df_temp)
#print(y_t)
##將驗證圖片讀檔，二值化並存成2維陣列180*(4096)
size = 128
img_tra = []

# 指定要列出所有檔案的目錄
mypath = "./Training"
# 取得所有檔案與子目錄名稱
files = listdir(mypath)
# 以迴圈處理
for i,f in enumerate(files):
    # 產生檔案的絕對路徑
    fullpath = join(mypath, f)
    #print(f)
    # 判斷 fullpath 是檔案還是目錄
    ro = False
    if isfile(fullpath):
        im = cv2.resize(cv2.imread(fullpath,0),(size,size))
        image = cv2.resize(cv2.imread(fullpath),(size,size))
        image_array=np.array(image/255,dtype='float32')
        img_tra.append(image_array)
img_tra = np.array(img_tra)
print(img_tra.shape)
print(y_t.shape)

(1080, 128, 128, 3)
(1080, 18)


### 驗證/測試集img&label

In [9]:
##將cvs裡面答案轉為陣列
df_v = pd.read_excel('./Validation-label.xlsx',header=None)  
#print(df_v)
#df_v = df_v.T
df_v_temp = pd.get_dummies(df_v[0]) #One-hot編碼
y_v = np.array(df_v_temp)

img_val = []#np.zeros((50,size*size,3))
# 指定要列出所有檔案的目錄
mypath = "./Validation"
# 取得所有檔案與子目錄名稱
files = listdir(mypath)
for i,f in enumerate(files):
    # 產生檔案的絕對路徑
    fullpath = join(mypath, f)
    #print(f)
    # 判斷 fullpath 是檔案還是目錄
    ro = False
    if isfile(fullpath):
        im = cv2.resize(cv2.imread(fullpath,0),(size,size))
        image = cv2.resize(cv2.imread(fullpath),(size,size))
        img_val.append(image/255)
img_val = np.array(img_val)
print(img_val.shape)
print(y_v.shape)

(50, 128, 128, 3)
(50, 18)


In [10]:
from keras.utils import np_utils
from keras.datasets import mnist
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.models import Sequential
img_pix=size
#建立一個線性堆疊的模型
model = Sequential()
#建立卷積層1
model.add(Conv2D(filters=32,kernel_size=(5,5),padding='same',input_shape=(img_pix,img_pix,3),activation='relu'))
#建立池化層1。
model.add(MaxPooling2D(pool_size=(2,2)))
#建立卷積層2
model.add(Conv2D(filters=64,kernel_size=(5,5),padding='same',activation='relu'))
#建立池化層2
model.add(MaxPooling2D(pool_size=(2,2)))
#避免overfitting
model.add(Dropout(0.25))
#建立平滑層
model.add(Flatten())
#建立隱藏層，且避免overfitting
model.add(Dense(img_pix,activation='relu'))
model.add(Dropout(0.15))
#建立輸出層
model.add(Dense(18,activation='softmax'))
#設定損失函數，最佳化方法，以及評估模型等。
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()
##開始訓練
train_history=model.fit(x=img_tra ,y=y_t,validation_data=(img_val,y_v),\
                        epochs=30,batch_size=150,verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              